# KG-Hub Tutorial 3 - Link Prediction and More Graph Machine Learning

This notebook serves as a practical guide to advanced KG-Hub features and resources. A brief review of the Getting Started tutorial notebook is not a strict prerequisite but may be helpful.  

This notebook also assumes you are in a Linux environment, but Google Colab is an option as well.

Here's an example question for our use case: which foods may impact DNA repair pathways? It's a broad question with many possible answers, or no answers at all. A KG may hold some clues. We don't want to be entirely reliant on existing data, however: starting with sets of chemicals, foods, and biological pathways, we can perform link prediction to predict additional connections.

## Setup

First, we'll install the requirements.


In [ ]:
!pip install kgx
!pip install kghub-downloader

In [ ]:
import os
import yaml
import pandas as pd

Now we need to set up two things for KGX to work properly:
* A download config file
* A merge config file

In practice, we may need to write a new transform for each new source, but all of the sources we'll use here are conveniently already available as KGX node and edge files on KG-Hub.

We'll download five sources. Two are ontologies available through the KG-OBO project on KG-Hub: FOODON, a food ontology, and CHEBI, a chemical ontology. The other sources are sets of preprocessed [Reactome](https://reactome.org) pathways, connections between those pathways, and mappings between those pathways and chemicals. They're all defined in a dictionary below, with the name of each source as its key and a list of one or more source URLs as its value. We've also defined a set of local filenames, as we know what the compressed ontology files should contain.

In [ ]:
data_dir = "./" # Just the current directory, though in practice it would be something like data/raw/
sources = {"foodon":["https://kg-hub.berkeleybop.io/kg-obo/foodon/2022-02-01/foodon_kgx_tsv.tar.gz"],
           "chebi":["https://kg-hub.berkeleybop.io/kg-obo/chebi/210/chebi_kgx_tsv.tar.gz"],
           "chebi2reactome":["https://kg-hub.berkeleybop.io/kg-idg/20220601/transformed/reactome/chebi2reactome_edges.tsv",
                             "https://kg-hub.berkeleybop.io/kg-idg/20220601/transformed/reactome/chebi2reactome_nodes.tsv"],
           "reactome_pathways":["https://kg-hub.berkeleybop.io/kg-idg/20220601/transformed/reactome/reactomepathways_nodes.tsv"],
           "reactome_relations":["https://kg-hub.berkeleybop.io/kg-idg/20220601/transformed/reactome/reactomepathwaysrelation_edges.tsv"]}
local_filepaths = {"foodon":["foodon_kgx_tsv_edges.tsv",
                            "foodon_kgx_tsv_nodes.tsv"],
           "chebi":["chebi_kgx_tsv_edges.tsv",
                    "chebi_kgx_tsv_nodes.tsv"],
           "chebi2reactome":["chebi2reactome_edges.tsv",
                             "chebi2reactome_nodes.tsv"],
           "reactome_pathways":["reactomepathways_nodes.tsv"],
           "reactome_relations":["reactomepathwaysrelation_edges.tsv"]}

There is an example of a KGX download config file [here](https://github.com/Knowledge-Graph-Hub/kg-dtm-template/blob/master/download.yaml), but it's easy to assemble from scratch with something like the following:

In [ ]:
source_data = []
for source in sources:
  for url in sources[source]:
    local_name = url.rpartition('/')[-1]
    source_data.append({"url":url,
                        "local_name":local_name})

with open("download.yaml", "w") as dl_config:
  yaml.dump(source_data, dl_config, default_flow_style=False)

Now we may use the config file with the `kghub-downloader` to download all sources.

In [ ]:
!downloader download.yaml

Decompress the compressed sources.

In [ ]:
!cat *.tar.gz | tar zxvf - -i

Next step: set up a merge config file. Our sources are already in the expected KGX graph format, so no transformation is necessary.

See the [example merge config](https://github.com/Knowledge-Graph-Hub/kg-dtm-template/blob/master/merge.yaml) in this repository for further inspiration.

In [ ]:
merge_data = {"configuration":{"output_directory":data_dir,
                              "checkpoint":"false"
                              },
              "merged_graph":{"name":"tutorial_graph",
                              "source":{},
                              "operations":[{"name": "kgx.graph_operations.summarize_graph.generate_graph_stats",
                                        "args":{"graph_name":"tutorial_graph",
                                        "filename":"merged_graph_stats.yaml"
                                                }
                                                }
                                                ],
                                "destination":{"merged-kg-tsv":{"format":"tsv",
                                              "filename": "merged-kg"}
                                                },            
                                }
                }

for source in local_filepaths:
  merge_data["merged_graph"]["source"][source] = {"name":source,
                                                  "input":{"format":"tsv",
                                                          "filename":local_filepaths[source]}
                                                  }

with open("merge.yaml", "w") as merge_config:
  yaml.dump(merge_data, merge_config, default_flow_style=False)

## KG Assembly

The data files are all here and the configuration files are ready. We may now use `kgx` to assemble a single set of nodes and edges from them all.

In [ ]:
from kgx.cli.cli_utils import merge

In [ ]:
# This will take a moment. You've probably been sitting for too long - go take a walk for a few minutes.
merged_graph = merge("merge.yaml")

If everything went as expected, the merged KG will be in `merged-kg_edges.tsv` and `merged-kg_nodes.tsv`. There will also be a `merged_graph_stats.yaml` detailing the new graph contents. 

A brief aside - there's an alternative merge approach called `cat-merge`. [The package is here](https://github.com/monarch-initiative/cat-merge) and an example of its use may be found in KG-Bioportal [here](https://github.com/ncbo/kg-bioportal/blob/main/kg_bioportal/merge_utils/merge_kg.py).

Let's take a quick look at the stats file.

In [ ]:
with open("merged_graph_stats.yaml") as yaml_file:
    config = yaml.load(yaml_file, Loader=yaml.FullLoader)

In [ ]:
# Count of all edges in the graph
print(config["edge_stats"]["total_edges"])

# Count of all nodes in the graph
print(config["node_stats"]["total_nodes"])

In [ ]:
# What kind of nodes are in the graph?
for category in config["node_stats"]["node_categories"]:
    print(category)

Nodes in ontologies and data sources are assigned appropriate Biolink Model categories whenever possible. Those assigned `NamedThing` may still belong to a more detailed category, but assigning such a category may be challenging.

Now let's take a look at the graph contents to begin examining how they may answer our questions.

Let's get a set of all relations between food entries in FOODON and chemical entries in CHEBI.

In [ ]:
!grep FOODON merged-kg_edges.tsv | grep CHEBI

The "subject, predicate, and object" of each relation are found in the second, third, and fourth columns, respectively.
I'll save you some trouble: every relation like `CHEBI:XXXXX    biolink:subclass_of FOODON:03412972` is just saying "this chemical is a [food additive](https://www.ebi.ac.uk/ols/ontologies/foodon/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FFOODON_03412972)". There are several different *types* of relations in this set, however. We can get a quick idea about those types by looking at the `merged_graph_stats.yaml` KGX has prepared for us.

In [ ]:
!sed -n '/count_by_predicates/,/count_by_spo/p' merged_graph_stats.yaml

We can see, for example, that there are >93 thousand "participates_in" relations. Let's see what the participants in these graph edges are:

In [ ]:
!grep -A 1 "participates_in" merged_graph_stats.yaml

So these are our connections between chemicals and pathways. The same counts appear multiple times because each node may have more than one category (e.g., a ChemicalEntity is also a NamedThing).

Continue to the next section for some examples of how to learn more about the new graph.

## Loading Graphs with `GraPE`

The `GraPE` library includes a substantial array of tools for working with graph data, generating reports and plots about graph contents, and preparing graph representations. We'll start by loading the graph from the previous section, then we'll get more details about its contents.

In [ ]:
!pip install grape -U

In [ ]:
from grape import Graph

Once the next block completes, it will output a long text report about the graph's properties and a variety of its "topological oddities". These don't mean anything is intrinsically *wrong* with the graph - rather, they are features of the data we have used to construct the graph. In some cases, for example, a CHEBI entry may be present within our imported data despite being deleted from the dataset and therefore obsolete, so it will be among the singleton nodes. These oddities *may* have an impact on the value of the graph embeddings we'll assemble in the next section and they *may* highlight areas where our data is structured in ways we may not expect.

In [ ]:
g = Graph.from_csv(
  directed=False, # This is a directed graph - we're treating it as undirected to make it easier to examine paths.
  node_path='merged-kg_nodes.tsv',
  edge_path='merged-kg_edges.tsv',
  verbose=True,
  nodes_column='id',
  node_list_node_types_column='category',
  default_node_type='biolink:NamedThing',
  sources_column='subject',
  destinations_column='object',
  edge_list_edge_types_column='predicate',
  name="A Graph of Foods"
)
g

Now let's try to find all edges connecting nodes to a DNA repair pathway. The human DNA repair pathway as an ID of **R-HSA-73894** in Reactome, so in our graph it will have an identifier of **REACT:R-HSA-73894**. We can run the following to find all related edges:

In [ ]:
one_step = g.get_neighbour_node_names_from_node_name(node_name="REACT:R-HSA-73894")
one_step

So this pathway only has other Reactome pathways as neighbors. That's fine - we can check for neighbors of those neighbors.

In [ ]:
two_step = []
for neighbor in one_step:
    for result in g.get_neighbour_node_names_from_node_name(node_name=neighbor):
        two_step.append(result)
two_step

Now things are getting interesting. Several of these nodes are CHEBI entries. These aren't drugs or environmental contaiminants, though: they're participants in the pathways, like CHEBI:16991 - that's just [DNA](https://www.ebi.ac.uk/ols/ontologies/chebi/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCHEBI_16991).

We can see the whole path, in terms of the names of nodes in that path:

In [ ]:
g.get_shortest_path_node_names_from_node_names(src_node_name="REACT:R-HSA-73894", dst_node_name="CHEBI:456216")

Using the same strategy, we can see if there are other paths of interest.

Let's get a set of all FOODON entries and see if any of them have paths back to the DNA Repair pathway node. This will take a short while. We're also most interested in the _shortest_ paths, so we'll sort the paths by length and look at the shortest ones.

In [ ]:
food_nodes = []
for result in g.get_node_names():
    if result.startswith("FOODON"):
        food_nodes.append(result)

print(f"Found {len(food_nodes)} nodes from FOODON.")

food_paths = []
for node in food_nodes:
    try:
        path = g.get_shortest_path_node_names_from_node_names(src_node_name="REACT:R-HSA-73894", dst_node_name=node)
        food_paths.append(path)
    except ValueError:
        # This means there isn't a path to the destination from the specified node
        continue

food_paths.sort(key=len)
print(food_paths[0:9])

You can expect to find a few paths of length 5, at least. I'll spoil one of them:

`['REACT:R-HSA-73894', 'REACT:R-HSA-73942', 'REACT:R-HSA-73943', 'CHEBI:16526', 'FOODON:03301011']`
is a path from the DNA Repair pathway to the DNA Damage Reversal pathway to the "Reversal of alkylation damage by DNA dioxygenases" pathway which involves carbon dioxide (CHEBI:16526) as a component, as do carbonated beverages (FOODON:03301011). 

## Graph Embeddings

GraPE is particularly efficient at preparing graph embeddings. 
Let's see a list of its available node embedding methods:

In [ ]:
from grape import get_available_models_for_node_embedding
get_available_models_for_node_embedding()

Let's use Ensmallen's implementation of a method named [HOPE](https://www.kdd.org/kdd2016/papers/files/rfp0184-ouA.pdf). For the sake of simplicity, we'll specify the metric as 'Adjacency' below. We'll also set `enable_cache` to true so the embeddings get saved locally (in the current working directory, under `/embedding/HOPE/Ensmallen/[name of the graph]`.

In [ ]:
from grape.embedders import HOPEEnsmallen

Remove disconnected nodes first, as they won't contribute much to our embeddings and may cause errors.

In [ ]:
g = g.remove_disconnected_nodes()

In [ ]:
model = HOPEEnsmallen(metric="Adjacency",enable_cache=True)
embedding = model.fit_transform(g)

Now let's see what those embeddings look like. They won't be too informative just yet.

In [ ]:
embedding.get_node_embedding_from_index(0)

Now we'll create some plots. This may take a minute or two.

In [ ]:
from grape import GraphVisualizer
visualizer = GraphVisualizer(g)

In [ ]:
visualizer.fit_and_plot_all(embedding)

These plots primarily serve to show how the graph contents may be separated based on the newly created embeddings. Ideally, we'd like there to be a clear feature distinguishing existing vs non-existing edges, or it will be difficult to predict which new edges may be likely to exist. Different embeddings and metrics are likely to correlate with potentially useful features, but there is no guarantee that any one feature will be sufficient for consistently predicting new edges.

We can pass an embedding method by name, too - try the plot specified below and compare with the results above. Or, continue to the next section to see how to predict new edges.

In [ ]:
visualizer = GraphVisualizer(g)
visualizer.fit_and_plot_all("NetMF",iterations=1,walk_length=5,embedding_size=10)

## Edge Prediction

We may finally try to predict some new edges between foods and pathways. As we saw above, there are no paths directly between only these two types of nodes. Instead, we can attempt to find new potential relationships between all nodes in the graph.

At this point we'll just need to work with the largest fully connected component of the graph, as our methods won't work if all our nodes aren't connected to each other in some way. Let's see how many components are in the graph right now:

In [ ]:
g.get_number_of_connected_components()

The displayed triple tells us the total number of components, the size of the smallest component in nodes, and the size of the largest component in nodes, respectively. Most importantly, there's more than one component in there. Let's trim it down to just the component containing our pathway of interest.

In [ ]:
trim_g = g.remove_components(node_names=['REACT:R-HSA-73894'])
trim_g.get_number_of_connected_components()

That's better - we should have just one component now. We have a few options regarding next steps:
*  We can use the node embeddings to train a model and apply the model to generate predicted edges. This will rely upon the topology of the graph. In practice, NEAT will run this entire pipeline, from graph loading to edge prediction, based on a single config file (see the Machine Learning on Knowledge Graphs tutorial notebook for an example). We'll skip that here as we've already done some of the work.
* We can get node features based on their text with a BERT model.
* We can use a node feature such as its category. Our graph doesn't have too many different sources and we haven't specified detailed categories, so this may not be very informative here.
* We can use edge features.

First, let's see some of the edge prediction methods we can work with.

In [ ]:
from grape import get_available_models_for_edge_prediction
get_available_models_for_edge_prediction()

### Perceptron and Edge Features

The next step will evaluate how well Ensmallen's Perceptron model implementation works on edge prediction. Note that this doesn't include any embedding-derived features.

In [ ]:
from embiggen.edge_prediction.edge_prediction_ensmallen.perceptron import PerceptronEdgePrediction

We assemble train and test subgraphs, with an 80/20 split.

In [ ]:
train, test = trim_g.connected_holdout(train_size=0.8)

Get an embedding of the training graph. As before, this may take some time.

In [ ]:
embedding = HOPEEnsmallen(metric="Adjacency").fit_transform(train)

Next, we train the model. Also a potentially time-consuming process, but I assume you're patient and/or have other things to occupy your time.

In [ ]:
model = PerceptronEdgePrediction(
    edge_features=None,
    number_of_edges_per_mini_batch=16,
    edge_embeddings="CosineSimilarity"
)
model.fit(
    graph=train, 
    node_features=embedding
)

With the newly trained model, we may now provide it the nodes in our test set and see how good it is at predicting them. Each of these edges already exist, so we expect to see many high confidence scores.

In [ ]:
test_pred = model.predict_proba(
    graph=test,
    node_features=embedding,
    return_predictions_dataframe=True
)
hist = test_pred.hist(bins=100,
                       color="darkgreen",
                       column="predictions", 
                       xlabelsize = 14,
                       ylabelsize = 14,
                      )

That's not really *prediction*, though. Let's predict some new edges! To do so, we just need a negative set of potential edges, then repeat the process as before.

In [ ]:
negative_graph = trim_g.sample_negative_graph(number_of_negative_samples=test.get_number_of_edges())
pred = model.predict_proba(
    graph=negative_graph,
    node_features=embedding,
    return_predictions_dataframe=True
)
hist = pred.hist(bins=100,
                       color="orange",
                       column="predictions", 
                       xlabelsize = 14,
                       ylabelsize = 14,
                      )

There's a possibility that some of our negative edges are in the original graph since we did some trimming. Let's remove all the existing edges from the results and sort them.

In [ ]:
cols = ["sources", "destinations"]

# Remove all existing edges
all_edge_node_names = trim_g.get_edge_node_names(directed=False)
pred = pred[~pred[cols].apply(tuple, 1).isin(all_edge_node_names)]

# Remove self-interactions
pred = pred[pred['sources'] != pred['destinations']]

# Sort
pred.sort_values(by=['predictions'], ascending=False, inplace=True)
pred

These scores may not be as high as they could be, so some further optimization may be in order. 

Before we forget about our use case, let's see which of these predictions involve one of the 1-hop or 2-hop neighbors of our focus pathway, REACT:R-HSA-73894.

In [ ]:
neighbors = one_step + two_step
pred = pred[(pred['sources'].str.contains('|'.join(neighbors)))|(pred['destinations'].str.contains('|'.join(neighbors)))]
pred

If you see `FOODON:03309530`, that's the entry for "muffin". I'm afraid I won't be able to explain how or why a muffin may impact DNA repair.

We may also filter by prefix when obtaining predictions:

In [ ]:
pred = model.predict_proba_bipartite_graph_from_edge_node_prefixes(
    graph=negative_graph,
    node_features=embedding,
    return_predictions_dataframe=True,
    source_node_prefixes=["REACT"],
    destination_node_prefixes=["CHEBI"]
)

cols = ["sources", "destinations"]

# Process as before.
all_edge_node_names = trim_g.get_edge_node_names(directed=False)
pred = pred[~pred[cols].apply(tuple, 1).isin(all_edge_node_names)]

pred = pred[pred['sources'] != pred['destinations']]

pred.sort_values(by=['predictions'], ascending=False, inplace=True)
pred

### Text Embeddings

Let's see if the names and descriptions of the nodes in our graph contain features useful for edge prediction.

In [ ]:
from grape.datasets import get_okapi_tfidf_weighted_textual_embedding

In [ ]:
embedding = get_okapi_tfidf_weighted_textual_embedding('merged-kg_nodes.tsv',
                                                       separator = "\t",
                                                       header = True,
                                                      columns=["name","description"],
                                                      verbose = True)
embedding

We convert the embeddings to a dataframe so they can be mapped to the subset of nodes in our trimmed graph.

In [ ]:
bert_df = pd.DataFrame(embedding, index=g.get_node_names())
subgraph_bert_df = bert_df.loc[trim_g.get_node_names()]

Now let's plot those BERT embeddings to see what we're dealing with.

In [ ]:
visualizer = GraphVisualizer(trim_g)
visualizer.fit_and_plot_all(subgraph_bert_df)

The results are not obviously useful. There isn't a clear distinction between existing and non-existing edges. That being said, we can still pass them on to a model to see if we can get any useful edge predictions.